In [ ]:
from manim import *
import random

class SVMintro(Scene):
    def construct(self):
        # ---------- 坐标系（仅第一象限） ----------
        axes_left = Axes(
            x_range=[0, 5, 1],
            y_range=[0, 5, 1],
            x_length=5,
            y_length=5,
            tips=True,
        ).move_to(DOWN)
        x_label_left = axes_left.get_x_axis_label(Tex("X"))
        y_label_left = axes_left.get_y_axis_label(Tex("Y"))
        # ---------- 生成左侧两簇点 ----------
        cluster1_left = VGroup()
        cluster2_left = VGroup()
        random.seed(42)
        for _ in range(10):
            x1 = 1.5 + random.uniform(-0.4, 0.5)
            y1 = 1.5 + random.uniform(-0.5, 0.9)
            dot1 = Dot(axes_left.coords_to_point(x1, y1), color=BLUE)
            cluster1_left.add(dot1)
            x2 = 3.5 + random.uniform(-0.5, 0.8)
            y2 = 3.5 + random.uniform(-0.6, 0.5)
            dot2 = Dot(axes_left.coords_to_point(x2, y2), color=RED)
            cluster2_left.add(dot2)
        # ---------- 左侧直线 ----------
        p1 = axes_left.coords_to_point(1.3, 4.5)
        p2 = axes_left.coords_to_point(4, 0)
        line_left = Line(p1, p2, color=YELLOW_D, stroke_width=3)
        # 左侧标题
        title_left = Text("理想情况", font="Microsoft YaHei", font_size=60, color=GREEN_C)
        title_left.next_to(axes_left, UP, buff=0.5).shift(LEFT*3.6)
        # ---------- 左侧组合 ----------
        scatter_left = VGroup(axes_left, x_label_left, y_label_left, cluster1_left, cluster2_left, line_left)
        scatter_left.shift(DOWN*5).scale(0.9)
        self.play(scatter_left.animate.shift(UP*4.85),
                  run_time=1.5)
        # ---------- 决策边界箭头与标注 ----------
        mid_x = (1.3 + 4) / 2
        mid_y = (4.5 + 0) / 2
        mid_point = axes_left.coords_to_point(mid_x, mid_y)
        arrow = Arrow(
            start=mid_point + LEFT*0.5 + DOWN*0.5,
            end=mid_point + RIGHT*0.5 + UP*0.5,
            color=YELLOW_D,
            buff=0
        ).shift(RIGHT*1.5+DOWN*1.0)
        label = Text("决策边界", font="Microsoft YaHei", font_size=28, color=YELLOW_D)
        label.next_to(arrow, RIGHT, buff=0.2).shift(UP*1.0+LEFT*1.0)
        self.play(FadeIn(arrow, label))
        # ---------- 新点 ----------
        new_dot = Dot(axes_left.coords_to_point(3, 1.2), color=WHITE, radius=0.1)
        self.play(FadeIn(new_dot))
        gg1 = VGroup(scatter_left, arrow, label, new_dot)
        self.play(gg1.animate.shift(LEFT*3.9))
        self.play(FadeIn(title_left))
        self.wait(5)

        # ---------- 右侧散点图（点更分散） ----------
        axes_right = axes_left.copy().next_to(axes_left, RIGHT, buff=2.5)
        x_label_right = axes_right.get_x_axis_label(Tex("X"))
        y_label_right = axes_right.get_y_axis_label(Tex("Y"))
        cluster1_right = VGroup()
        cluster2_right = VGroup()
        random.seed(100)  # 改随机种子保证点不同
        for _ in range(10):
            x1 = 1.5 + random.uniform(-0.99, 0.99)  # 更分散
            y1 = 1.5 + random.uniform(-0.9, 1.9)
            cluster1_right.add(Dot(axes_right.coords_to_point(x1, y1), color=BLUE))
            x2 = 3.5 + random.uniform(-1.5, 1.5)
            y2 = 3.5 + random.uniform(-0.99, 0.9)
            cluster2_right.add(Dot(axes_right.coords_to_point(x2, y2), color=RED))
        p1_r = axes_right.coords_to_point(1.3, 4.5)
        p2_r = axes_right.coords_to_point(4, 0)
        line_right = Line(p1_r, p2_r, color=YELLOW_D, stroke_width=3)
        # 右侧标题
        title_right = Text("实际情况", font="Microsoft YaHei", font_size=60, color=RED_C)
        title_right.next_to(axes_right, UP, buff=0.5).shift(UP*0.3)
        # ---------- 右侧自定义决策边界 ----------
        # 自定义右侧直线起点和终点（在右侧坐标系内的坐标）
        p1_custom = axes_right.coords_to_point(0, 3.25)
        p2_custom = axes_right.coords_to_point(5, 1.85)
        line_right_custom = Line(p1_custom, p2_custom, color=YELLOW_D, stroke_width=3)

        # 替换原来的右侧线
        scatter_right = VGroup(axes_right, x_label_right, y_label_right, cluster1_right, cluster2_right, line_right_custom)
        self.play(FadeIn(scatter_right), FadeIn(title_right))
        self.wait(5)
        # ---------- 移除原右侧点和线 ----------
        self.play(
            FadeOut(cluster1_right),
            FadeOut(cluster2_right),
            FadeOut(line_right_custom)
        )

        # ---------- 定义波浪线（2~3个起伏） ----------
        wave_curve = VMobject(color=YELLOW_D, stroke_width=3)
        wave_points = []
        num_waves = 3  # 3个完整起伏
        for i in range(0, 51):
            x = i / 10  # x 从0到5
            y = 2.5 + 1.5 * np.sin(num_waves * np.pi * x / 5)  # 波幅1.5，高起伏
            wave_points.append(axes_right.coords_to_point(x, y))
        wave_curve.set_points_smoothly(wave_points)

        # ---------- 根据波浪线生成两类点（数量变多） ----------
        cluster1_wave = VGroup()
        cluster2_wave = VGroup()
        np.random.seed(300)
        num_points = 40  # 每类40个点
        for _ in range(num_points):
            x_pixel = np.random.uniform(0.1, 4.9)
            y_wave = 2.5 + 1.5 * np.sin(num_waves * np.pi * x_pixel / 5)
            # 蓝色点在波浪线上方
            y1 = y_wave + np.random.uniform(0.3, 0.7)
            cluster1_wave.add(Dot(axes_right.coords_to_point(x_pixel, y1), color=BLUE))
            # 红色点在波浪线下方
            y2 = y_wave - np.random.uniform(0.3, 0.7)
            cluster2_wave.add(Dot(axes_right.coords_to_point(x_pixel, y2), color=RED))

        # ---------- 添加到场景 ----------
        self.play(FadeIn(cluster1_wave), FadeIn(cluster2_wave))
        self.wait(2)
        self.play(FadeIn(wave_curve))
        self.wait(5)

        right2 = VGroup(cluster1_wave, cluster2_wave, wave_curve, axes_right, x_label_right, y_label_right, title_right)
        left2 = VGroup(scatter_left, title_left, arrow, label, new_dot)
        t1 = Text("SVM", font="Microsoft YaHei", font_size=140, color=GREEN_C)
        t1.move_to(UR).shift(RIGHT*2+DOWN*1)
        t1.shift(RIGHT*0.8)
        t1.set_opacity(0)
        self.play(
            left2.animate.shift(LEFT*7),
            right2.animate.shift(LEFT*6.5),
            t1.animate.shift(LEFT*0.8).set_opacity(1),
            run_time=1.5
        )
        self.wait(2)